In [ ]:
# --------------------------------------------
# Modernized Rescue Dashboard (Bootstrap UI)
# --------------------------------------------

from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64

from crud import AnimalShelter


# ---------------------------
# Database Connection / Setup
# ---------------------------
db = AnimalShelter()

def load_data():
    df = pd.DataFrame.from_records(db.read({}))
    if "_id" in df.columns:
        df.drop(columns=["_id"], inplace=True)
    return df

df = load_data()


# ---------------------------
# App Setup with Bootstrap
# ---------------------------
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.COSMO])

encoded_image = base64.b64encode(open("logo.png", "rb").read()).decode()


# ---------------------------
# Layout
# ---------------------------
app.layout = dbc.Container([
    # Title Row
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H1("SNHU CS-340 Rescue Dashboard", className="text-center"),
                ])
            ])
        ])
    ], className="mt-4"),

    # Logo Row
    dbc.Row([
        dbc.Col([
            html.Img(src=f"data:image/png;base64,{encoded_image}", height="120px")
        ], className="text-center")
    ]),

    # Filter Row
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Select Rescue Type"),
                dbc.CardBody([
                    dbc.RadioItems(
                        id="filter-type",
                        options=[
                            {"label": "Water Rescue", "value": "WR"},
                            {"label": "Mountain/Wilderness Rescue", "value": "MWR"},
                            {"label": "Disaster / Individual Tracking", "value": "DRIT"},
                            {"label": "Reset", "value": "RESET"},
                        ],
                        value="RESET",
                        inline=True,
                    )
                ])
            ])
        ])
    ], className="mt-3"),

    # Table Row
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Animal Records"),
                dbc.CardBody([
                    dash_table.DataTable(
                        id="datatable-id",
                        columns=[{"name": c, "id": c} for c in df.columns],
                        data=df.to_dict("records"),
                        filter_action="native",
                        sort_action="native",
                        page_action="native",
                        page_size=10,
                        row_selectable="single",
                        style_table={'overflowX': 'auto'},
                        style_header={'fontWeight': 'bold'},
                        style_cell={'padding': '8px'},
                    )
                ])
            ])
        ])
    ], className="mt-4"),

    # Graph + Map Row
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Breed Distribution"),
                dbc.CardBody(id="graph-id")
            ])
        ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Animal Location"),
                dbc.CardBody(id="map-id")
            ])
        ], md=6)
    ], className="mt-4 mb-5")
])



# ---------------------------
# Callbacks
# ---------------------------

# FILTER TABLE
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    df = load_data()

    if filter_type == "WR":
        query = {
            "$and": [
                {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
                {"sex_upon_outcome": "Intact Female"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
        df = pd.DataFrame.from_records(db.read(query))

    elif filter_type == "MWR":
        query = {
            "$and": [
                {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                                   "Siberian Husky", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
        df = pd.DataFrame.from_records(db.read(query))

    elif filter_type == "DRIT":
        query = {
            "$and": [
                {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                                   "Bloodhound", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
            ]
        }
        df = pd.DataFrame.from_records(db.read(query))

    if "_id" in df.columns:
        df.drop(columns=["_id"], inplace=True)

    return df.to_dict("records")



# PIE CHART
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if not viewData:
        return dcc.Graph(figure={})

    dff = pd.DataFrame(viewData)
    fig = px.pie(dff, names='breed', title='Animals by Breed')
    fig.update_traces(textinfo='percent+label')
    fig.update_layout(margin=dict(t=40, b=0, l=0, r=0))

    return dcc.Graph(figure=fig)



# MAP
@app.callback(
    Output('map-id', "children"),
    [
        Input('datatable-id', "derived_virtual_data"),
        Input('datatable-id', "derived_virtual_selected_rows")
    ]
)
def update_map(viewData, selected):
    if not viewData:
        return "No data available."

    dff = pd.DataFrame(viewData)

    if not selected:
        return "Select a row to view location."

    row = dff.iloc[selected[0]]

    lat = row.get("location_lat", 30.75)
    lon = row.get("location_long", -97.48)

    return dl.Map(
        center=[lat, lon],
        zoom=12,
        style={'width': '100%', 'height': '450px'},
        children=[
            dl.TileLayer(),
            dl.Marker(
                position=[lat, lon],
                children=[
                    dl.Tooltip(row.get("breed", "")),
                    dl.Popup([
                        html.H4(row.get("name", "")),
                        html.P(f"Breed: {row.get('breed')}"),
                        html.P(f"ID: {row.get('animal_id')}")
                    ])
                ]
            )
        ]
    )


app.run_server(debug=True)
